1. Carga de dataset
Se eligió el siguiente dataset que contiene información de vuelos, pasajeros, rutas y aerolíneas en el país desde el año 2017.
Es provista por la Subsecretaría de Turismo.
https://datos.gob.ar/dataset/turismo-conectividad-aerea/archivo/turismo_aab49234-28c9-48ab-a978-a83485139290

In [27]:
# Se debe bajar el dataset base_microdatos.csv y colocarla en la misma carpeta del proyecto
import pandas as pd

# Cargar el CSV en un dataframe (se crea carpeta datos para mejor organización)
df_vuelos = pd.read_csv("datos/base_microdatos.csv")

# Mostrar las primeras filas para verificar carga correcta
df_vuelos.head()

###pip install sqlalchemy pymysql python-dotenv

,indice_tiempo,clasificacion_vuelo,clase_vuelo,aerolinea,origen_oaci,origen_aeropuerto,origen_localidad,origen_provincia,origen_pais,origen_continente,destino_oaci,destino_aeropuerto,destino_localidad,destino_provincia,destino_pais,destino_continente,pasajeros,asientos,vuelos
0,2017-01-01,Cabotaje,No Regular,Andes Líneas Aéreas,SAAV,Aeropuerto de Sauce Viejo,Santa Fe,Santa Fe,Argentina,América del Sur,SAZS,Aeropuerto Int. Tte. Luis Candelaria,San Carlos de Bariloche,Río Negro,Argentina,América del Sur,85,85,1
1,2017-01-01,Cabotaje,No Regular,Andes Líneas Aéreas,SASA,Aeropuerto Int. Martín Miguel de Güemes,Salta,Salta,Argentina,América del Sur,SASJ,Aeropuerto Int. Gdor. Horacio Guzmán,San Salvador de Jujuy,Jujuy,Argentina,América del Sur,23,83,1
2,2017-01-01,Cabotaje,No Regular,Andes Líneas Aéreas,SAZS,Aeropuerto Int. Tte. Luis Candelaria,San Carlos de Bariloche,Río Negro,Argentina,América del Sur,SAAV,Aeropuerto de Sauce Viejo,Santa Fe,Santa Fe,Argentina,América del Sur,78,83,1
3,2017-01-01,Cabotaje,Regular,Aerolíneas Argentinas,SAAR,Aeropuerto Int. de Rosario Islas Malvinas,Rosario,Santa Fe,Argentina,América del Sur,SACO,Aeropuerto Int. Ingeniero Ambrosio Taravella,Córdoba,Córdoba,Argentina,América del Sur,48,128,1
4,2017-01-01,Cabotaje,Regular,Aerolíneas Argentinas,SAAR,Aeropuerto Int. de Rosario Islas Malvinas,Rosario,Santa Fe,Argentina,América del Sur,SAEZ,Aeropuerto Int. Ministro Pistarini,Ezeiza,Buenos Aires,Argentina,América del Sur,90,128,1


2. Almacenamiento en base de datos
Se escogió MySQL Workbench 8.0 Community (cliente) junto a XAMPP 3.0 (servidor)
Para poder acceder a la base de datos, se creará un archivo .env el cual almacenará variables de entorno (como credenciales)
Esto será más seguro ya que protege al mantenerlas fuera del código fuente.

In [28]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text

# Cargar el archivo .env y almacenarlas en variables
load_dotenv(override=True)

db_user=os.getenv("DB_USER")
db_pass=os.getenv("DB_PASS")
db_host=os.getenv("DB_HOST")
db_port=os.getenv("DB_PORT")
db_name=os.getenv("DB_NAME")
tb_name=os.getenv("TB_NAME")

# Crear conexión al servidor MySQL (sin base de datos)
connection_url_no_db = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}"
engine = create_engine(connection_url_no_db)

# Crear la base de datos si no existe
with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {db_name}"))
    print(f"✅ Base de datos '{db_name}' creada o ya existente.")

# Al usar with se evita olvidar cerrar la conexión con conn.close().

# Crear engine ahora apuntando a la base de datos
connection_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine_db = create_engine(connection_url)

# Verificar conexión a la base de datos creada
with engine_db.connect() as conn:
    result = conn.execute(text("SELECT DATABASE();"))
    print("🎯 Conectado a la base de datos:", result.scalar())

# Crear tabla si no existe
with engine_db.connect() as conn:
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {tb_name} (
    indice_tiempo DATETIME,
    clasificacion_vuelo VARCHAR(50),
    clase_vuelo VARCHAR(50),
    aerolinea VARCHAR(100),
    origen_oaci VARCHAR(10),
    origen_aeropuerto VARCHAR(100),
    origen_localidad VARCHAR(100),
    origen_provincia VARCHAR(100),
    origen_pais VARCHAR(100),
    origen_continente VARCHAR(100),
    destino_oaci VARCHAR(10),
    destino_aeropuerto VARCHAR(100),
    destino_localidad VARCHAR(100),
    destino_provincia VARCHAR(100),
    destino_pais VARCHAR(100),
    destino_continente VARCHAR(100),
    pasajeros INT,
    asientos INT,
    vuelos INT
    )
    """
    conn.execute(text(create_table_query))
    print(f"Tabla '{tb_name}' creada (si no existía).")

# Pasar datos del dataframe a la tabla de MySQL
    df_vuelos.to_sql(
    name=tb_name,         # nombre de la tabla en la base de datos
    con=engine_db,
    if_exists='replace',  # 'replace' borra la tabla si existe, 'append' agrega filas (sólo cargaremos por unica vez)
    index=False           # para no guardar el índice del DataFrame como columna
    )

    print(f"Tabla '{tb_name}' cargada con datos.")


✅ Base de datos 'db_aereos' creada o ya existente.
🎯 Conectado a la base de datos: db_aereos
Tabla 'tb_vuelos' creada (si no existía).


3. Construcción del dashboard
Se usará notebook de jupyter para crear dashboard con la información de la base de datos.
Se eligió Plotly para la visualización de gráficos.

In [39]:
#pip install plotly
#pip install nbformat
#pip install --upgrade --force-reinstall nbformat

import pandas as pd
import plotly.express as px

# Cargar datos de MySQL a dataframe
df_vuelos_sql = pd.read_sql("SELECT * FROM tb_vuelos", con=engine_db)

### TOP destinos más visitados

# Agrupar por destino y sumar pasajeros
destino_mas_visitado = df_vuelos_sql.groupby(["destino_localidad","destino_pais"])["pasajeros"].sum().sort_values(ascending=False)

# Mostrar el top 1
print("🛬 Destino más visitado:")
print(destino_mas_visitado.head(1))

# Top 10 de los más visitados
top_destinos = destino_mas_visitado.head(10).reset_index()
top_destinos["destino_etiqueta"] = top_destinos["destino_localidad"] + " (" + top_destinos["destino_pais"] + ")"

# Convertir cantidad pasajeros en millones
top_destinos["pasajeros_millones"] = (top_destinos["pasajeros"] / 1_000_000).round(3)


fig = px.bar(
    top_destinos,
    x="destino_etiqueta",
    y="pasajeros_millones",
    title="✈️ Top 10 destinos más visitados",
    color="pasajeros_millones",
    text="pasajeros_millones"
)
fig.update_layout(xaxis_title="Destino (País)", yaxis_title="Pasajeros (millones)")
fig.show()

🛬 Destino más visitado:
destino_localidad       destino_pais
Ciudad de Buenos Aires  Argentina       50179619
Name: pasajeros, dtype: int64


In [ ]:
### Aerolíneas con más y menos vuelos

# Agrupar por aerolínea y cantidad de vuelos
vuelos_por_aerolinea = df_vuelos_sql.groupby("aerolinea")["vuelos"].sum().sort_values(ascending=False)

print("🛫 Aerolínea con más vuelos:")
print(vuelos_por_aerolinea.head(1))

print("\n🛬 Aerolínea con menos vuelos:")
print(vuelos_por_aerolinea.tail(1))

fig = px.bar(
    vuelos_por_aerolinea.reset_index(),
    x="aerolinea",
    y="vuelos",
    title="✈️ Total de vuelos por aerolínea",
    color="vuelos",
    text="vuelos"
)
fig.show()

🛫 Aerolínea con más vuelos:
aerolinea
Aerolíneas Argentinas    836387
Name: vuelos, dtype: int64

🛬 Aerolínea con menos vuelos:
aerolinea
TAME    1
Name: vuelos, dtype: int64


In [ ]:
###

# Asegurar tipo datetime
df_vuelos_sql["indice_tiempo"] = pd.to_datetime(df_vuelos_sql["indice_tiempo"])

# Filtrar por año
df_2025 = df_vuelos[df_vuelos["indice_tiempo"].dt.year == 2025]

# Agrupar por aerolínea
vuelos_por_aerolinea = df_2025.groupby("aerolinea")["vuelos"].sum().reset_index()

# Calcular el total y el porcentaje
total_vuelos = vuelos_por_aerolinea["vuelos"].sum()
vuelos_por_aerolinea["porcentaje"] = vuelos_por_aerolinea["vuelos"] / total_vuelos * 100

# Agrupar aerolíneas con < 1% como 'Otras'
vuelos_por_aerolinea["aerolinea_agrupada"] = vuelos_por_aerolinea.apply(
    lambda row: row["aerolinea"] if row["porcentaje"] >= 1 else "Otras",
    axis=1
)

# Reagrupar los datos para el gráfico final
df_final = vuelos_por_aerolinea.groupby("aerolinea_agrupada")["vuelos"].sum().reset_index()

# Pie chart
fig = px.pie(
    df_final,
    names="aerolinea_agrupada",
    values="vuelos",
    title="🥧 Participación de aerolíneas en el total de vuelos (2025)",
    hole=0.3
)

fig.show()